In [ ]:
import ee

# Initialize
ee.Initialize(project='ee-howardwanghr')

print("Searching for your assets...")

# Method 1: Check the standard Cloud Assets folder
try:
    assets = ee.data.listAssets({'parent': 'projects/ee-howardwanghr/assets'})
    print("\n--- FOUND THESE FILES ---")
    if 'assets' in assets:
        for asset in assets['assets']:
            print(f"COPY THIS ID ->  {asset['name']}")
    else:
        print("(No files found in the main 'assets' folder)")
except Exception as e:
    print(f"Error checking cloud folder: {e}")

# Method 2: Check for Legacy Assets (Just in case it's stored under 'users')
# Sometimes files live here instead
try:
    # We guess the username from the project ID
    legacy_path = 'users/howardwanghr' 
    assets = ee.data.listAssets({'parent': legacy_path})
    print(f"\n--- FOUND IN LEGACY FOLDER ({legacy_path}) ---")
    if 'assets' in assets:
        for asset in assets['assets']:
            print(f"COPY THIS ID ->  {asset['name']}")
except:
    pass

In [ ]:
import ee
import geemap
import ipywidgets as widgets
from ipyleaflet import WidgetControl

# 1. INITIALIZE & LOAD DATA
try:
    ee.Initialize(project='ee-howardwanghr')
except:
    ee.Authenticate()
    ee.Initialize(project='ee-howardwanghr')

# Load your assets
aoni = ee.FeatureCollection("projects/ee-howardwanghr/assets/dorset_aoni")
h3_grid = ee.FeatureCollection("projects/ee-howardwanghr/assets/dorset_h3_real")

# 2. GLOBAL STATE (Stores selected IDs)
selected_h3_ids = []

# 3. CREATE MAP
m = geemap.Map(center=[50.7, -2.4], zoom=10)

# Add Base Grid (Blue)
style_base = {'color': '4285F4', 'width': 1, 'fillColor': '4285F411'}
m.add_layer(h3_grid.style(**style_base), {}, 'H3 Hex Grid')

# 4. CREATE THE DASHBOARD WIDGET (Replaces ui.Panel)
# We use HTML for the content so we can style it easily
inspector_output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '10px'})
title_widget = widgets.HTML("<h3>🕵️ MULTI-HEX INSPECTOR</h3>")
status_widget = widgets.HTML("Selected Hexagons: <b>0</b>")
clear_btn = widgets.Button(description="❌ Clear Selection", button_style='danger', layout={'display': 'none'})

# Container for the results (Overlap alerts)
results_widget = widgets.VBox([])

# Combine them into a sidebar panel
panel = widgets.VBox([title_widget, status_widget, clear_btn, inspector_output, results_widget])
panel.layout.width = '350px'
panel.layout.background_color = 'rgba(255, 255, 255, 0.95)'

# Add the panel to the Map as a floating control
control = WidgetControl(widget=panel, position='bottomright')
m.add_control(control)

# 5. CLEAR BUTTON LOGIC
def clear_selection(b):
    global selected_h3_ids
    selected_h3_ids = [] # Empty list
    status_widget.value = "Selected Hexagons: <b>0</b>"
    status_widget.style.text_color = 'gray'
    clear_btn.layout.display = 'none'
    results_widget.children = [] # Clear results
    
    # Remove layers
    if m.find_layer('Selected Hexagons'):
        m.remove_layer(m.find_layer('Selected Hexagons'))
    if m.find_layer('Overlapping Areas'):
        m.remove_layer(m.find_layer('Overlapping Areas'))

clear_btn.on_click(clear_selection)

# 6. THE BRAINS (Update Map & Panel)
def update_view():
    # Update Status Text
    count = len(selected_h3_ids)
    status_widget.value = f"Selected Hexagons: <b>{count}</b>"
    status_widget.style.text_color = 'black' if count > 0 else 'gray'
    
    # Show/Hide Clear Button
    clear_btn.layout.display = 'block' if count > 0 else 'none'
    
    # Remove old layers if they exist
    if m.find_layer('Selected Hexagons'):
        m.remove_layer(m.find_layer('Selected Hexagons'))
    if m.find_layer('Overlapping Areas'):
        m.remove_layer(m.find_layer('Overlapping Areas'))

    if count == 0:
        results_widget.children = []
        return

    # --- GEOSPATIAL LOGIC ---
    # Filter grid to get selected hexes
    selected_hexes = h3_grid.filter(ee.Filter.inList('h3_index', selected_h3_ids))
    
    # Add Highlight Layer (Yellow)
    style_highlight = {'color': 'FFFF00', 'width': 3, 'fillColor': 'FFFF0022'}
    m.add_layer(selected_hexes.style(**style_highlight), {}, 'Selected Hexagons')

    # Find Overlaps
    # Note: Union() combines them into one geometry for faster checking
    combined_geom = selected_hexes.geometry() 
    overlaps = aoni.filterBounds(combined_geom)

    # Add Overlap Layer (Red)
    style_overlap = {'color': 'FF0000', 'width': 2, 'fillColor': 'FF000044'}
    m.add_layer(overlaps.style(**style_overlap), {}, 'Overlapping Areas')

    # --- UPDATE RESULTS PANEL ---
    # We use getInfo() here to pull data to Python (be careful with large data!)
    results_widget.children = [widgets.HTML("<i>Calculating overlaps...</i>")]
    
    try:
        overlap_list = overlaps.getInfo()['features']
        
        children = [widgets.HTML("<hr>")] # Separator
        
        if len(overlap_list) > 0:
            # ALERT BOX
            alert_html = f"<div style='background:red; color:white; padding:5px; font-weight:bold'>⚠️ {len(overlap_list)} PROTECTED AREAS FOUND</div>"
            children.append(widgets.HTML(alert_html))
            
            # LIST NAMES
            for feat in overlap_list:
                props = feat['properties']
                # Try different property names just in case
                name = props.get('Name') or props.get('NAME') or props.get('desig') or "Unnamed Feature"
                children.append(widgets.HTML(f"<div style='margin-left:10px'>• {name}</div>"))
        else:
            # SAFE BOX
            safe_html = "<div style='background:green; color:white; padding:5px; font-weight:bold'>✅ NO OVERLAP DETECTED</div>"
            children.append(widgets.HTML(safe_html))
            children.append(widgets.Label(f"None of the {count} hexes touch a protected area."))
            
        results_widget.children = children

    except Exception as e:
        results_widget.children = [widgets.Label(f"Error: {e}")]

# 7. CLICK HANDLER
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        # Get coordinates from the click event
        lat, lon = kwargs.get('coordinates')
        point = ee.Geometry.Point([lon, lat])
        
        # Find which hex was clicked
        # We assume the grid is dense, so filterBounds is efficient
        clicked_hex = h3_grid.filterBounds(point).first()
        
        # We need to get the ID back to Python to update our list
        # computations inside getInfo() are blocking, but necessary here
        try:
            hex_info = clicked_hex.getInfo()
            if hex_info:
                h_id = hex_info['properties']['h3_index']
                
                # Toggle Logic
                if h_id in selected_h3_ids:
                    selected_h3_ids.remove(h_id)
                else:
                    selected_h3_ids.append(h_id)
                
                # Update UI
                update_view()
                
        except Exception as e:
            # Clicked outside the grid or error
            pass

# Attach the click handler to the map
m.on_interaction(handle_click)

# Display Map
m

In [ ]:
import ee
import geemap
import h3
import ipywidgets as widgets
from ipyleaflet import WidgetControl

# 1. INITIALIZE
try:
    ee.Initialize(project='ee-howardwanghr')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-howardwanghr')

# Load your Protected Areas
protected_areas = ee.FeatureCollection("projects/ee-howardwanghr/assets/dorset_aoni")

# 2. SETUP MAP
m = geemap.Map(center=[50.7, -2.4], zoom=11)

# 3. DEFINE ZOOM LOGIC
def get_h3_res(zoom_level):
    """Decides hexagon size based on zoom level"""
    if zoom_level >= 13: return 10
    if zoom_level >= 11: return 9
    if zoom_level >= 9:  return 8
    return 7

# 4. CREATE THE DYNAMIC FUNCTION
def update_grid(**kwargs):
    bounds = m.bounds
    zoom = m.zoom
    if not bounds: return
    
    south, west = bounds[0]
    north, east = bounds[1]
    
    poly_geojson = {
        'type': 'Polygon',
        'coordinates': [[
            [west, north], [east, north], [east, south], [west, south], [west, north]
        ]]
    }
    
    current_res = get_h3_res(zoom)
    try:
        hex_ids = h3.polygon_to_cells(poly_geojson, current_res)
        
        if len(hex_ids) > 2000:
            hex_ids = list(hex_ids)[:2000]
            print(f"⚠️ Capped at 2000 hexes (Zoom in for more)")

        features = []
        for h_id in hex_ids:
            boundary = h3.cell_to_boundary(h_id)
            flipped = [[p[1], p[0]] for p in boundary]
            geom = ee.Geometry.Polygon([flipped])
            features.append(ee.Feature(geom, {'h3_index': h_id}))
            
        dynamic_layer = ee.FeatureCollection(features)
        
        layer_name = 'Dynamic H3 Grid'
        if m.find_layer(layer_name): 
            m.remove_layer(m.find_layer(layer_name))
            
        style = {'color': 'blue', 'width': 1, 'fillColor': '0000FF05'}
        m.add_layer(dynamic_layer.style(**style), {}, layer_name)
        
        info_label.value = f"<b>Resolution:</b> {current_res} | <b>Hexes in View:</b> {len(hex_ids)}"
        
    except Exception as e:
        print(f"Grid Error: {e}")

# 5. UI CONTROLS
info_label = widgets.HTML("<b>Status:</b> Move map to generate grid.")
refresh_btn = widgets.Button(description="🔄 Refresh Grid", button_style='info')
refresh_btn.on_click(lambda b: update_grid())

panel = widgets.VBox([info_label, refresh_btn])
control = WidgetControl(widget=panel, position='bottomright')
m.add_control(control)

# 6. CLICK INSPECTOR (FIXED)
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        
        res = get_h3_res(m.zoom)
        click_id = h3.latlng_to_cell(lat, lon, res)
        
        boundary = h3.cell_to_boundary(click_id)
        flipped = [[p[1], p[0]] for p in boundary]
        click_geom = ee.Geometry.Polygon([flipped])
        
        overlaps = protected_areas.filterBounds(click_geom)
        count = overlaps.size().getInfo()
        
        if m.find_layer('Selected'): m.remove_layer(m.find_layer('Selected'))
        if m.find_layer('Revealed Data'): m.remove_layer(m.find_layer('Revealed Data'))
        
        # --- FIX 1: USE HEX CODES (So '44' transparency works) ---
        color = 'FF0000' if count > 0 else '00FF00'  # FF0000=Red, 00FF00=Green
        
        feat = ee.Feature(click_geom, {})
        
        # --- FIX 2: WRAP IN COLLECTION ---
        fc_selected = ee.FeatureCollection([feat])
        
        # Now 'color' + '44' becomes 'FF000044' (Valid) instead of 'red44' (Invalid)
        m.add_layer(fc_selected.style(**{'color': color, 'width': 3, 'fillColor': color + '44'}), {}, 'Selected')
        
        if count > 0:
            clipped = overlaps.map(lambda f: f.intersection(click_geom, 1))
            m.add_layer(clipped.style(color='red', width=2), {}, 'Revealed Data')
            info_label.value = f"⚠️ <b>Constraint Found:</b> {count} areas inside Hex {click_id}"
        else:
             info_label.value = f"✅ <b>Safe:</b> Hex {click_id} is clear."

m.on_interaction(handle_click)

# Display
m

In [ ]:
# Diagnostic: Check properties in National Habitat Networks dataset
import ee
ee.Initialize(project='ee-howardwanghr')

habitat_networks = ee.FeatureCollection("projects/ee-howardwanghr/assets/National_Habitat_Networks_England")

# Get first feature and print its properties
sample = habitat_networks.first()
print("Habitat Network Properties:")
print(sample.getInfo()['properties'])

# Get a few features to see the data
samples = habitat_networks.limit(1000).getInfo()['features']
print("\nFirst 20 features:")
for i, feat in enumerate(samples):
    print(f"\nFeature {i+1} properties:")
    for key, value in feat['properties'].items():
        print(f"  {key}: {value}")


In [ ]:
# Check sample coordinates to confirm the CRS mismatch
import ee
ee.Initialize(project='ee-howardwanghr')

sssi_test = ee.FeatureCollection("projects/ee-howardwanghr/assets/Sites_of_Special_Scientific_Interest_England")
protected_test = ee.FeatureCollection("projects/ee-howardwanghr/assets/dorset_aoni")

# Get first feature and check its CRS and coordinates
sssi_sample = sssi_test.first()
protected_sample = protected_test.first()

print("SSSI Projection:", sssi_sample.geometry().projection().getInfo())
print("Protected Areas Projection:", protected_sample.geometry().projection().getInfo())

# Get sample coordinates to see the actual values
sssi_coords = sssi_sample.geometry().bounds().getInfo()['coordinates'][0][0]
protected_coords = protected_sample.geometry().bounds().getInfo()['coordinates'][0][0]

print("\nSSI Sample Coordinates:", sssi_coords)
print("Protected Sample Coordinates:", protected_coords)
print("\n⚠️ If SSSI coordinates are 5-6 digits (like 400000, 100000), they're in British National Grid")
print("   but labeled as EPSG:4326. This causes the rightward shift.")


In [ ]:
# SSSI Data Issue Diagnosis
# 
# The coordinates ARE in WGS84 format (not British National Grid), but they appear
# shifted on the map. This means the source data has incorrect coordinates.
#
# To fix this properly:
# 1. Download official SSSI data from: https://naturalengland-defra.opendata.arcgis.com/
#    Search for "Sites of Special Scientific Interest (England)"
# 2. Verify the CRS in QGIS or similar tool
# 3. If needed, reproject to EPSG:4326 before uploading
# 4. Re-upload to Earth Engine with correct coordinates
#
# For now, SSSI layer is hidden on map but still works for overlap detection.
# The overlap detection works because coordinates are internally consistent.

print("✓ SSSI coordinates are in correct format (WGS84)")
print("✗ But the actual coordinate values appear to be shifted")
print("→ This is a data quality issue from the original upload")
print("\nRecommendation: Re-upload SSSI data from official Natural England source")


In [ ]:
%pip install h3

In [ ]:
import ee
import geemap
import h3
import ipywidgets as widgets
from ipyleaflet import WidgetControl, DrawControl
import json

# 1. INITIALIZE EARTH ENGINE
try:
    ee.Initialize(project='ee-howardwanghr')
except Exception:
    ee.Authenticate()
    ee.Initialize(project='ee-howardwanghr')

# Load Protected Data and Agricultural Land Classification
protected_areas = ee.FeatureCollection("projects/ee-howardwanghr/assets/dorset_aoni")

# SSSI data - use as-is for now (coordinate shift issue needs to be fixed at upload)
sssi_areas = ee.FeatureCollection("projects/ee-howardwanghr/assets/Sites_of_Special_Scientific_Interest_England")

# Special Areas of Conservation (SAC) and Special Protection Areas (SPA)
sac_areas = ee.FeatureCollection("projects/ee-howardwanghr/assets/Special_Areas_of_Conservation")
spa_areas = ee.FeatureCollection("projects/ee-howardwanghr/assets/Special_Protection_Areas")

# Ancient Woodland
ancient_woodland = ee.FeatureCollection("projects/ee-howardwanghr/assets/ancient-woodland")

# Ramsar Sites and National Nature Reserves
ramsar_sites = ee.FeatureCollection("projects/ee-howardwanghr/assets/Ramsar_England")
national_nature_reserves = ee.FeatureCollection("projects/ee-howardwanghr/assets/National_Nature_Reserves")

# BNG Opportunity Enhancement Datasets
national_habitat_networks = ee.FeatureCollection("projects/ee-howardwanghr/assets/National_Habitat_Networks_England")
chalk_rivers = ee.FeatureCollection("projects/ee-howardwanghr/assets/Chalk_Rivers_England_6056247281574748187")

agri_land = ee.FeatureCollection("projects/ee-howardwanghr/assets/England_Agricultural_Land_class")

# 2. SETUP MAP
m = geemap.Map(
    center=[50.7, -2.4], 
    zoom=12,
    draw_control=False,  # We'll add our own custom draw control
    toolbar_control=False,  # Remove duplicate toolbar
    layer_control=True  # Keep layer control
)

# Add satellite imagery as base layer
m.add_basemap('SATELLITE')

# Add protected areas layer (dark red style for satellite visibility)
m.add_layer(
    protected_areas.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'Protected Areas (AONI)',
    True,  # shown by default
    0.5    # slightly increased opacity
)

# Add SSSI layer (same dark red style as protected areas)
# NOTE: SSSI has coordinate shift issue - hidden by default
# The data still works for analysis, just displays shifted on the map
m.add_layer(
    sssi_areas.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'SSSI (England) - ⚠️ Coordinate Shift',
    False,  # HIDDEN by default due to coordinate shift
    0.5     # slightly increased opacity
)

# Add Special Areas of Conservation (SAC) layer
# NOTE: SAC has coordinate shift issue - hidden by default
# The data still works for analysis, just displays shifted on the map
m.add_layer(
    sac_areas.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'Special Areas of Conservation (SAC) - ⚠️ Coordinate Shift',
    False,  # HIDDEN by default due to coordinate shift
    0.5     # slightly increased opacity
)

# Add Special Protection Areas (SPA) layer
# NOTE: SPA has coordinate shift issue - hidden by default
# The data still works for analysis, just displays shifted on the map
m.add_layer(
    spa_areas.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'Special Protection Areas (SPA) - ⚠️ Coordinate Shift',
    False,  # HIDDEN by default due to coordinate shift
    0.5     # slightly increased opacity
)

# Add Ancient Woodland layer
m.add_layer(
    ancient_woodland.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'Ancient Woodland',
    True,  # shown by default
    0.5    # slightly increased opacity
)

# Add Ramsar Sites layer
# NOTE: Ramsar has coordinate shift issue - hidden by default
# The data still works for analysis, just displays shifted on the map
m.add_layer(
    ramsar_sites.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'Ramsar Sites - ⚠️ Coordinate Shift',
    False,  # HIDDEN by default due to coordinate shift
    0.5     # slightly increased opacity
)

# Add National Nature Reserves layer
# NOTE: NNR has coordinate shift issue - hidden by default
# The data still works for analysis, just displays shifted on the map
m.add_layer(
    national_nature_reserves.style(color='8B0000', width=1.5, fillColor='8B000020'),
    {},
    'National Nature Reserves - ⚠️ Coordinate Shift',
    False,  # HIDDEN by default due to coordinate shift
    0.5     # slightly increased opacity
)

# Add National Habitat Networks layer (BNG opportunity enhancement)
m.add_layer(
    national_habitat_networks.style(color='00AA00', width=0.5, fillColor='00AA0015'),
    {},
    'National Habitat Networks',
    False,  # hidden by default, shown during analysis
    0.3
)

# Add Chalk Rivers layer (BNG opportunity enhancement)
m.add_layer(
    chalk_rivers.style(color='0088FF', width=2),
    {},
    'Chalk Rivers',
    False,  # hidden by default, shown during analysis
    0.7
)

# Optional: Add ALC layer (initially hidden - will show overlapping areas during analysis)
# This layer will be dynamically updated to show only relevant ALC polygons

# --- HELPER 1: DATA CLEANER ---
def clean_numpy(data):
    """
    Forcefully removes Numpy types by round-tripping through JSON.
    """
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if hasattr(obj, 'tolist'): return obj.tolist()
            if hasattr(obj, 'item'): return obj.item()
            return super().default(obj)
    return json.loads(json.dumps(data, cls=NumpyEncoder))

# --- HELPER 2: BNG OPPORTUNITY SCORING (3-FACTOR MODEL) ---
def calculate_location_score(habitat_class):
    """
    Calculate location points based on habitat network classification.
    Returns: 1-10 points
    """
    if not habitat_class or habitat_class == 'No Data':
        return 1  # Background default
    
    class_lower = str(habitat_class).lower().strip()
    
    # Exact string matches (case-insensitive)
    if 'network enhancement zone 1' in class_lower or 'nez1' in class_lower:
        return 10  # Priority expansion zone
    elif 'network enhancement zone 2' in class_lower or 'nez2' in class_lower:
        return 7  # Buffer/improvement zone
    elif 'fragmentation action zone' in class_lower:
        return 7  # Critical repair zone
    elif 'restorable habitat' in class_lower:
        return 5  # High restoration potential
    elif 'network expansion zone' in class_lower:
        return 4  # Long-term connectivity
    else:
        return 1  # Default/Background

def calculate_soil_score(alc_grade):
    """
    Calculate soil feasibility points based on ALC grade.
    Returns: 0-5 points
    """
    if not alc_grade or alc_grade == 'No Data':
        return 0
    
    grade_lower = str(alc_grade).lower().strip()
    
    # Exact string matches (case-insensitive)
    if 'grade 5' in grade_lower or grade_lower == '5':
        return 5  # Poor quality - ideal for habitat
    elif 'grade 4' in grade_lower or grade_lower == '4':
        return 5  # Poor quality - ideal for habitat
    elif 'grade 3' in grade_lower or grade_lower == '3' or '3b' in grade_lower or '3a' in grade_lower:
        return 2  # Moderate quality
    elif 'grade 2' in grade_lower or grade_lower == '2':
        return 0  # High quality - preserve for agriculture
    elif 'grade 1' in grade_lower or grade_lower == '1':
        return 0  # Excellent quality - preserve for agriculture
    elif 'non-agricultural' in grade_lower or 'urban' in grade_lower:
        return 0  # Not suitable
    else:
        return 0  # Unknown - conservative

def calculate_water_bonus(has_chalk_river):
    """
    Calculate water proximity bonus.
    Returns: 0 or 3 points
    """
    # Note: We're using presence/absence since exact distance calculation
    # would require rasterization. If within 50m buffer, feature would overlap.
    return 3 if has_chalk_river else 0

def calculate_bng_score(alc_grade, habitat_class, has_chalk_river):
    """
    Calculate BNG Habitat Creation Potential as a percentage (0-100%).
    Formula: (Location Points + Soil Points + Water Points) / 18 × 100
    
    Returns: (percentage, rating_text, color_hex, score_loc, score_soil, score_water)
    Perfect Site: 100% (Zone 1 + Grade 5 + Chalk River = 18/18)
    """
    score_loc = calculate_location_score(habitat_class)
    score_soil = calculate_soil_score(alc_grade)
    score_water = calculate_water_bonus(has_chalk_river)
    
    # Calculate raw total
    total_raw = score_loc + score_soil + score_water
    
    # Convert to percentage (0-100%)
    percentage = round((total_raw / 18) * 100, 1)
    
    # Map to color gradient: Purple → Red → Orange → Yellow → Green → Blue
    # Percentage thresholds (warm colors = high potential, cool colors = low potential)
    if percentage >= 83.3:  # 15-18 pts (≥83.3%)
        return (percentage, 'Exceptional', '9933ff', score_loc, score_soil, score_water)  # Purple
    elif percentage >= 66.7:  # 12-14 pts (66.7-83.2%)
        return (percentage, 'Very High', 'ff3333', score_loc, score_soil, score_water)  # Red
    elif percentage >= 50.0:  # 9-11 pts (50-66.6%)
        return (percentage, 'High', 'ff8800', score_loc, score_soil, score_water)  # Orange
    elif percentage >= 33.3:  # 6-8 pts (33.3-49.9%)
        return (percentage, 'Moderate', 'ffcc00', score_loc, score_soil, score_water)  # Yellow
    elif percentage >= 16.7:  # 3-5 pts (16.7-33.2%)
        return (percentage, 'Low', '00cc66', score_loc, score_soil, score_water)  # Green
    else:  # 0-2 pts (0-16.6%)
        return (percentage, 'Very Low', '0066ff', score_loc, score_soil, score_water)  # Blue

# --- HELPER 3: HABITAT NETWORK CLASSIFICATION EXPLANATIONS ---
def get_habitat_classification_explanation(classification):
    """Returns explanation for habitat network classifications"""
    explanations = {
        # Habitat Components (Current State)
        'Primary Habitat': '🟢 <b>Core network patches</b> - Existing priority habitat areas that form the foundation of the network.',
        'Associated Habitat': '🟡 <b>Supporting habitats</b> - Natural companion habitats that support species from the primary habitat.',
        'Restorable Habitat': '🟠 <b>High restoration potential</b> - Degraded areas with existing habitat fragments, ideal for restoration.',
        'Habitat Creation': '🔵 <b>Active creation sites</b> - Areas where habitat creation is already underway (agri-environment schemes).',
        
        # Specific Habitat Types
        'Ancient woodland': '🌳 <b>Ancient Woodland - Primary Habitat</b> - Historic woodland that has existed continuously since at least 1600. These are irreplaceable habitats with high biodiversity value and form the core of woodland habitat networks. Ideal for protection and as source areas for woodland expansion.',
        
        # Network Zones (Future Action Areas)
        'Network Enhancement Zone 1': '⭐ <b>Priority expansion zone</b> - Prime land for re-creating primary habitat to connect existing patches. Soil/hydrology match target habitat needs.',
        'NEZ1': '⭐ <b>Priority expansion zone</b> - Prime land for re-creating primary habitat to connect existing patches.',
        'Network Enhancement Zone 2': '💚 <b>Buffer/improvement zone</b> - Less suitable for primary habitat but valuable for green infrastructure and wider improvements.',
        'NEZ2': '💚 <b>Buffer/improvement zone</b> - Less suitable for primary habitat but valuable for green infrastructure.',
        'Fragmentation Action Zone': '🚨 <b>Critical repair zone</b> - Emergency priority to reduce isolation of vulnerable, fragmented patches.',
        'Network Expansion Zone': '🌍 <b>Long-term connectivity</b> - Wider landscape opportunities to link separate habitat clusters across regions.'
    }
    return explanations.get(classification, f'ℹ️ <b>{classification}</b> - Priority habitat network area with high BNG potential.')

# --- HELPER 3: ROBUST FEATURE CREATION ---
def get_h3_res(zoom):
    z = int(zoom)
    if z >= 14: return 10
    if z >= 12: return 9
    return 8

def create_ee_feature(h_id):
    """
    Creates an ee.Feature WITHOUT passing a dict to the constructor.
    This bypasses the 'Unrecognized type: <class 'dict'>' error.
    """
    h_id = str(h_id)
    
    # 1. Get Boundary & Clean it
    raw_boundary = h3.cell_to_boundary(h_id)
    clean_boundary = clean_numpy(raw_boundary)
    
    # 2. Flip (Lat, Lon) -> (Lon, Lat)
    flipped = [[p[1], p[0]] for p in clean_boundary]
    
    # 3. Close the Ring
    if flipped[0] != flipped[-1]:
        flipped.append(flipped[0])
        
    # 4. Create Geometry
    geom = ee.Geometry.Polygon([flipped])
    
    # 5. Create Feature WITHOUT Dict
    # We create the feature with just geometry first.
    feat = ee.Feature(geom)
    
    # 6. Add Property via .set()
    # This avoids passing a dictionary object to the constructor.
    feat = feat.set('h3_index', h_id)
    
    return feat

# 3. ANALYSIS LOGIC
def run_analysis(drawn_geometry):
    # Hide instructions once processing starts
    info_label.layout.display = 'none'
    
    progress_label.value = (
        "<div style='padding:8px; background:#fff3cd; border-radius:4px; border-left:3px solid #ffc107;'>"
        "<b>⏳ Processing...</b><br>"
        "<span style='font-size:11px;'>Initializing analysis</span>"
        "</div>"
    )
    
    # Show the panel when analysis starts
    panel.layout.display = 'flex'
    
    # Clear old layers and results
    for layer in ['Safe Areas', 'Constraints Found', 'Constraint Borders', 'ALC Overlapping', 
                  'Habitat Networks (Overlapping)', 'Chalk Rivers (Overlapping)']:
        if m.find_layer(layer): m.remove_layer(m.find_layer(layer))
    
    results_panel.children = []  # Clear previous results

    try:
        # A. Sanitize Input
        clean_geo = clean_numpy(drawn_geometry)
        
        # B. Convert to H3 v4 Format with LatLngPoly
        geojson_coords = clean_geo['coordinates'][0]  # Get first ring
        
        # Convert from [lon, lat] to [[lat, lon], ...]
        h3_coords = []
        for coord in geojson_coords:
            lat = float(coord[1])
            lon = float(coord[0])
            h3_coords.append([lat, lon])
        
        # C. Generate Hex IDs using LatLngPoly wrapper
        res = get_h3_res(m.zoom)
        h3_poly = h3.LatLngPoly(h3_coords)  # CRITICAL: Wrap in LatLngPoly!
        hex_ids = h3.polygon_to_cells(h3_poly, res)
        
        if not hex_ids:
            progress_label.value = (
                "<div style='padding:8px; background:#f8d7da; border-radius:4px; border-left:3px solid #dc3545;'>"
                "<b>⚠️ No hexagons generated</b><br>"
                "<span style='font-size:11px;'>Draw a larger area</span>"
                "</div>"
            )
            return

        # D. Create List of EE Features
        progress_label.value = (
            "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
            f"<b>⏳ Preparing {len(hex_ids)} hexagons...</b><br>"
            "<span style='font-size:11px;'>Creating Earth Engine features</span>"
            "</div>"
        )
        
        # We explicitly create a list of ee.Feature objects
        feature_list = [create_ee_feature(h_id) for h_id in list(hex_ids)]
        
        # E. Upload to Earth Engine
        hex_fc = ee.FeatureCollection(feature_list)

        # F. PROPER INTERSECTION CHECK
        # Check actual geometry intersection by calculating intersection area
        def check_overlap(feature):
            hex_geom = feature.geometry()
            
            # Get all constraint areas that might overlap (bounding box check - fast)
            protected_candidates = protected_areas.filterBounds(hex_geom)
            sssi_candidates = sssi_areas.filterBounds(hex_geom)
            sac_candidates = sac_areas.filterBounds(hex_geom)
            spa_candidates = spa_areas.filterBounds(hex_geom)
            ancient_woodland_candidates = ancient_woodland.filterBounds(hex_geom)
            ramsar_candidates = ramsar_sites.filterBounds(hex_geom)
            nnr_candidates = national_nature_reserves.filterBounds(hex_geom)
            
            # Calculate actual intersection area with each candidate
            def calc_intersection_area(protected_feature):
                protected_geom = protected_feature.geometry()
                intersection = hex_geom.intersection(protected_geom, ee.ErrorMargin(1))
                area = intersection.area(ee.ErrorMargin(1))
                return ee.Feature(None, {'intersection_area': area})
            
            # Map over all constraint area candidates and calculate intersection areas
            protected_intersection_results = protected_candidates.map(calc_intersection_area)
            protected_intersection_area = protected_intersection_results.aggregate_sum('intersection_area')
            
            sssi_intersection_results = sssi_candidates.map(calc_intersection_area)
            sssi_intersection_area = sssi_intersection_results.aggregate_sum('intersection_area')
            
            sac_intersection_results = sac_candidates.map(calc_intersection_area)
            sac_intersection_area = sac_intersection_results.aggregate_sum('intersection_area')
            
            spa_intersection_results = spa_candidates.map(calc_intersection_area)
            spa_intersection_area = spa_intersection_results.aggregate_sum('intersection_area')
            
            ancient_woodland_intersection_results = ancient_woodland_candidates.map(calc_intersection_area)
            ancient_woodland_intersection_area = ancient_woodland_intersection_results.aggregate_sum('intersection_area')
            
            ramsar_intersection_results = ramsar_candidates.map(calc_intersection_area)
            ramsar_intersection_area = ramsar_intersection_results.aggregate_sum('intersection_area')
            
            nnr_intersection_results = nnr_candidates.map(calc_intersection_area)
            nnr_intersection_area = nnr_intersection_results.aggregate_sum('intersection_area')
            
            # Sum up all intersection areas from all constraint sources
            total_intersection_area = ee.Number(protected_intersection_area).add(
                ee.Number(sssi_intersection_area)
            ).add(
                ee.Number(sac_intersection_area)
            ).add(
                ee.Number(spa_intersection_area)
            ).add(
                ee.Number(ancient_woodland_intersection_area)
            ).add(
                ee.Number(ramsar_intersection_area)
            ).add(
                ee.Number(nnr_intersection_area)
            )
            
            # If total intersection area > 0, then there's overlap
            has_overlap = total_intersection_area.gt(0)
            
            # Store as 1 (unsafe) or 0 (safe)
            return feature.set('is_unsafe', ee.Algorithms.If(has_overlap, 1, 0))

        progress_label.value = (
            "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
            f"<b>⏳ Checking {len(hex_ids)} hexagons...</b><br>"
            "<span style='font-size:11px;'>Analyzing constraint overlaps</span>"
            "</div>"
        )
        classified_hexes = hex_fc.map(check_overlap)

        # G. Split & Visualize
        unsafe_fc = classified_hexes.filter(ee.Filter.eq('is_unsafe', 1))
        safe_fc = classified_hexes.filter(ee.Filter.eq('is_unsafe', 0))
        
        # Get counts
        total = len(hex_ids)
        bad = unsafe_fc.size().getInfo()
        safe_count = total - bad
        
        # Store resolution for click handler
        analyzed_hexagons['resolution'] = res
        analyzed_hexagons['all_hexagons'] = []
        
        # For SAFE hexagons, analyze Agricultural Land Classification
        if safe_count > 0:
            progress_label.value = (
                "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
                f"<b>⏳ Analyzing {safe_count} safe hexagons...</b><br>"
                "<span style='font-size:11px;'>Checking Agricultural Land Classification</span>"
                "</div>"
            )
            
            def classify_alc(feature):
                """Check which ALC grades and BNG opportunities overlap with this hexagon"""
                hex_geom = feature.geometry()
                
                # Find all ALC polygons that intersect this hexagon
                intersecting_alc = agri_land.filterBounds(hex_geom)
                
                # Get all unique grades in this hexagon
                def get_grade(alc_feature):
                    return alc_feature.get('ALC_GRADE')
                
                grades = intersecting_alc.aggregate_array('ALC_GRADE').distinct()
                
                # Join all grades into a comma-separated string
                grades_str = grades.join(', ')
                
                # Check for National Habitat Networks overlap - use 'Class' property
                intersecting_habitat_networks = national_habitat_networks.filterBounds(hex_geom)
                has_habitat_network = intersecting_habitat_networks.size().gt(0)
                
                # Get habitat network classifications from 'Class' property
                habitat_classes = intersecting_habitat_networks.aggregate_array('Class').distinct()
                habitat_classes_str = habitat_classes.join(', ')
                
                # Check for Chalk Rivers overlap
                intersecting_chalk_rivers = chalk_rivers.filterBounds(hex_geom)
                has_chalk_river = intersecting_chalk_rivers.size().gt(0)
                
                return feature.set({
                    'alc_grades': grades_str,
                    'has_habitat_network': has_habitat_network,
                    'habitat_classes': habitat_classes_str,
                    'has_chalk_river': has_chalk_river
                })
            
            safe_fc_with_alc = safe_fc.map(classify_alc)
            
            # Get the combined geometry of all safe hexagons
            safe_combined_geom = safe_fc.geometry()
            
            # Show ONLY the ALC polygons that overlap with safe hexagons
            alc_overlapping = agri_land.filterBounds(safe_combined_geom)
            
            # Show ONLY the Habitat Network areas that overlap with safe hexagons
            habitat_networks_overlapping = national_habitat_networks.filterBounds(safe_combined_geom)
            
            # Show ONLY the Chalk Rivers that overlap with safe hexagons
            chalk_rivers_overlapping = chalk_rivers.filterBounds(safe_combined_geom)
            
            # Style ALC by grade with updated color coding
            def style_alc_by_grade(feature):
                grade = ee.String(feature.get('ALC_GRADE'))
                # Color based on BNG rating: Blue->Green->Yellow->Orange->Red
                color = ee.Algorithms.If(
                    grade.match('.*[45].*'),  # Grade 4 or 5
                    '0066cc',  # Blue - High BNG
                    ee.Algorithms.If(
                        grade.match('.*3b.*'),  # Grade 3b
                        '28a745',  # Green - Medium-High
                        ee.Algorithms.If(
                            grade.match('.*3a.*'),  # Grade 3a
                            'fd7e14',  # Orange - Low
                            ee.Algorithms.If(
                                grade.match('.*[12].*'),  # Grade 1 or 2
                                'dc3545',  # Red - Very Low
                                'ffc107'  # Yellow - Medium (default Grade 3)
                            )
                        )
                    )
                )
                return feature.set('display_color', color)
            
            alc_styled = alc_overlapping.map(style_alc_by_grade)
            m.add_layer(alc_styled.style(color='000000', width=0.5, fillColor='ffc10730'), {}, 'ALC Overlapping')
            
            # Add Habitat Networks overlapping with safe hexagons (green highlighting)
            if habitat_networks_overlapping.size().getInfo() > 0:
                m.add_layer(
                    habitat_networks_overlapping.style(color='00AA00', width=1.5, fillColor='00AA0035'),
                    {},
                    'Habitat Networks (Overlapping)',
                    True,  # shown by default
                    0.6
                )
            
            # Add Chalk Rivers overlapping with safe hexagons (blue highlighting)
            if chalk_rivers_overlapping.size().getInfo() > 0:
                m.add_layer(
                    chalk_rivers_overlapping.style(color='0088FF', width=3, fillColor='0088FF00'),
                    {},
                    'Chalk Rivers (Overlapping)',
                    True,  # shown by default
                    0.8
                )
            
            # Color-code safe hexagons by comprehensive BNG scoring
            # Using 3-factor model: (Location + Soil + Water) / 18 × 100 = Percentage
            # Gradient: Purple (Exceptional) → Red (Very High) → Orange (High) → Yellow (Moderate) → Green (Low) → Blue (Very Low)
            
            # Calculate BNG percentage for each hexagon using the comprehensive 3-factor model
            def add_bng_score(feature):
                """Calculate BNG Habitat Creation Potential as percentage (0-100%)"""
                # Get ALC grade
                alc_grades_str = ee.String(feature.get('alc_grades'))
                primary_grade = ee.Algorithms.If(
                    alc_grades_str.length().gt(0),
                    alc_grades_str.split(',').get(0),
                    'No Data'
                )
                
                # Get habitat class
                habitat_str = ee.String(feature.get('habitat_classes'))
                primary_habitat = ee.Algorithms.If(
                    habitat_str.length().gt(0),
                    habitat_str.split(',').get(0),
                    ''
                )
                
                # Get chalk river status
                has_chalk = feature.get('has_chalk_river')
                
                # Calculate location score (1-10 pts based on habitat network priority)
                score_location = ee.Algorithms.If(
                    ee.String(primary_habitat).match('.*Network Enhancement Zone 1.*'),
                    10,  # NEZ1 - Priority expansion zones (~55% contribution)
                    ee.Algorithms.If(
                        ee.String(primary_habitat).match('.*Network Enhancement Zone 2.*|.*Fragmentation Action Zone.*'),
                        7,  # NEZ2 / Fragmentation - Buffer/repair zones (~39% contribution)
                        ee.Algorithms.If(
                            ee.String(primary_habitat).match('.*Restorable Habitat.*'),
                            5,  # Restorable - High restoration potential (~28% contribution)
                            ee.Algorithms.If(
                                ee.String(primary_habitat).match('.*Network Expansion Zone.*'),
                                4,  # Expansion - Long-term connectivity (~22% contribution)
                                1  # Base score (Background/No Data) (~5% contribution)
                            )
                        )
                    )
                )
                
                # Calculate soil score (0-5 pts based on ALC - lower grade = higher BNG)
                # Simplified per scoring matrix: Grade 4/5 = 5pts, Grade 3 = 2pts, Grade 1/2 = 0pts
                score_soil = ee.Algorithms.If(
                    ee.String(primary_grade).match('.*[45].*'),
                    5,  # Grade 4/5 - poor agricultural land, excellent for BNG (+28% boost)
                    ee.Algorithms.If(
                        ee.String(primary_grade).match('.*3.*'),
                        2,  # Grade 3 (all variants) - moderate quality (+11% contribution)
                        0  # Grade 1/2 / Urban / No Data - preserve for agriculture (0%, drags down)
                    )
                )
                
                # Calculate water bonus (0-3 pts)
                score_water = ee.Algorithms.If(has_chalk, 3, 0)  # +17% "cherry on top"
                
                # Total raw score
                total_raw = ee.Number(score_location).add(ee.Number(score_soil)).add(ee.Number(score_water))
                
                # Convert to percentage (0-100%)
                bng_percentage = total_raw.divide(18).multiply(100)
                
                # Assign color based on percentage thresholds
                # Warm colors (high potential) to cool colors (low potential)
                color = ee.Algorithms.If(
                    bng_percentage.gte(83.3),
                    '9933ff',  # Purple - Exceptional (≥83.3%, 15-18 pts)
                    ee.Algorithms.If(
                        bng_percentage.gte(66.7),
                        'ff3333',  # Red - Very High (66.7-83.2%, 12-14 pts)
                        ee.Algorithms.If(
                            bng_percentage.gte(50.0),
                            'ff8800',  # Orange - High (50-66.6%, 9-11 pts)
                            ee.Algorithms.If(
                                bng_percentage.gte(33.3),
                                'ffcc00',  # Yellow - Moderate (33.3-49.9%, 6-8 pts)
                                ee.Algorithms.If(
                                    bng_percentage.gte(16.7),
                                    '00cc66',  # Green - Low (16.7-33.2%, 3-5 pts)
                                    '0066ff'  # Blue - Very Low (0-16.6%, 0-2 pts)
                                )
                            )
                        )
                    )
                )
                
                return feature.set({
                    'bng_percentage': bng_percentage,
                    'bng_raw_score': total_raw,
                    'bng_color': color
                })
            
            # Apply BNG scoring to all safe hexagons
            safe_fc_scored = safe_fc_with_alc.map(add_bng_score)
            
            # Filter and display by percentage ranges
            # Exceptional BNG (Purple) - ≥83.3% (15-18 pts)
            exceptional_bng = safe_fc_scored.filter(ee.Filter.gte('bng_percentage', 83.3))
            if exceptional_bng.size().getInfo() > 0:
                m.add_layer(exceptional_bng.style(color='9933ff', width=1.5, fillColor='9933ffBB'), {}, 'Exceptional BNG (≥83%, Purple)')
            
            # Very High BNG (Red) - 66.7-83.2% (12-14 pts)
            very_high_bng = safe_fc_scored.filter(ee.Filter.And(ee.Filter.gte('bng_percentage', 66.7), ee.Filter.lt('bng_percentage', 83.3)))
            if very_high_bng.size().getInfo() > 0:
                m.add_layer(very_high_bng.style(color='ff3333', width=1.5, fillColor='ff3333BB'), {}, 'Very High BNG (67-83%, Red)')
            
            # High BNG (Orange) - 50-66.6% (9-11 pts)
            high_bng = safe_fc_scored.filter(ee.Filter.And(ee.Filter.gte('bng_percentage', 50.0), ee.Filter.lt('bng_percentage', 66.7)))
            if high_bng.size().getInfo() > 0:
                m.add_layer(high_bng.style(color='ff8800', width=1.5, fillColor='ff8800BB'), {}, 'High BNG (50-67%, Orange)')
            
            # Moderate BNG (Yellow) - 33.3-49.9% (6-8 pts)
            moderate_bng = safe_fc_scored.filter(ee.Filter.And(ee.Filter.gte('bng_percentage', 33.3), ee.Filter.lt('bng_percentage', 50.0)))
            if moderate_bng.size().getInfo() > 0:
                m.add_layer(moderate_bng.style(color='ffcc00', width=1.5, fillColor='ffcc00BB'), {}, 'Moderate BNG (33-50%, Yellow)')
            
            # Low BNG (Green) - 16.7-33.2% (3-5 pts)
            low_bng = safe_fc_scored.filter(ee.Filter.And(ee.Filter.gte('bng_percentage', 16.7), ee.Filter.lt('bng_percentage', 33.3)))
            if low_bng.size().getInfo() > 0:
                m.add_layer(low_bng.style(color='00cc66', width=1.5, fillColor='00cc66BB'), {}, 'Low BNG (17-33%, Green)')
            
            # Very Low BNG (Blue) - 0-16.6% (0-2 pts)
            very_low_bng = safe_fc_scored.filter(ee.Filter.lt('bng_percentage', 16.7))
            if very_low_bng.size().getInfo() > 0:
                m.add_layer(very_low_bng.style(color='0066ff', width=1.5, fillColor='0066ffBB'), {}, 'Very Low BNG (0-17%, Blue)')
        
        # Visualize constrained hexagons (DARK GREY - not suitable for development)
        m.add_layer(unsafe_fc.style(color='4a4a4a', width=2, fillColor='4a4a4a55'), {}, 'Constraints Found')
        
        # Show protected area boundaries if there are overlaps (dark red outline)
        # NOTE: SSSI, SAC, SPA, Ramsar, and NNR are excluded from visualization due to coordinate shift, but still used in analysis
        if bad > 0:
            combined = unsafe_fc.geometry()
            real_overlap_protected = protected_areas.filterBounds(combined)
            # real_overlap_sssi = sssi_areas.filterBounds(combined)  # Excluded from visualization
            # real_overlap_sac = sac_areas.filterBounds(combined)  # Excluded from visualization
            # real_overlap_spa = spa_areas.filterBounds(combined)  # Excluded from visualization
            real_overlap_ancient_woodland = ancient_woodland.filterBounds(combined)
            # real_overlap_ramsar = ramsar_sites.filterBounds(combined)  # Excluded from visualization
            # real_overlap_nnr = national_nature_reserves.filterBounds(combined)  # Excluded from visualization
            # Merge constraint layers (excluding SSSI, SAC, SPA, Ramsar, and NNR due to coordinate shift)
            real_overlap = real_overlap_protected.merge(real_overlap_ancient_woodland)
            m.add_layer(real_overlap.style(color='8B0000', width=2.5, fillColor='8B000030'), {}, 'Constraint Borders')

        # H. DETAILED STATS & INFO DISPLAY
        pct = int((bad / total) * 100) if total > 0 else 0
        
        # Analyze ALC for safe hexagons (needed for both cases)
        safe_fc_with_alc = None
        if safe_count > 0:
            info_label.value = f"⏳ Analyzing {safe_count} safe hexagons for BNG opportunity..."
            
            def classify_alc(feature):
                """Check which ALC grades and BNG opportunities overlap with this hexagon"""
                hex_geom = feature.geometry()
                intersecting_alc = agri_land.filterBounds(hex_geom)
                grades = intersecting_alc.aggregate_array('ALC_GRADE').distinct()
                grades_str = grades.join(', ')
                
                # Check for BNG opportunity enhancements - use 'Class' property
                intersecting_habitat_networks = national_habitat_networks.filterBounds(hex_geom)
                has_habitat_network = intersecting_habitat_networks.size().gt(0)
                
                # Get habitat network classifications
                habitat_classes = intersecting_habitat_networks.aggregate_array('Class').distinct()
                habitat_classes_str = habitat_classes.join(', ')
                
                intersecting_chalk_rivers = chalk_rivers.filterBounds(hex_geom)
                has_chalk_river = intersecting_chalk_rivers.size().gt(0)
                
                return feature.set({
                    'alc_grades': grades_str,
                    'has_habitat_network': has_habitat_network,
                    'habitat_classes': habitat_classes_str,
                    'has_chalk_river': has_chalk_river
                })
            
            safe_fc_with_alc = safe_fc.map(classify_alc)
        
        # Update status header
        if bad == 0:
            info_label.value = (
                "<div style='padding:10px; background:#d4edda; border-left:4px solid #28a745; border-radius:4px;'>"
                f"<b>✅ SITE CLEAR</b><br>"
                f"<span style='font-size:13px;'>All {total} hexagons are safe</span>"
                "</div>"
            )
        else:
            info_label.value = (
                f"<div style='padding:10px; background:#f8d7da; border-left:4px solid #dc3545; border-radius:4px;'>"
                f"<b>⚠️ {bad} OF {total} HEXAGONS CONSTRAINED ({pct}%)</b>"
                f"</div>"
            )
        
        # Build detailed results panel
        progress_label.value = (
            "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
            "<b>⏳ Generating report...</b><br>"
            "<span style='font-size:11px;'>Compiling detailed analysis</span>"
            "</div>"
        )
        
        # Get detailed information about unsafe hexagons and overlapping areas
        if bad > 0:
            # Get unsafe hexagon details (OPTIMIZED: limit for UI performance)
            display_limit = min(bad, 300)  # Limit to 300 hexagons for display
            unsafe_list = unsafe_fc.limit(display_limit).getInfo()['features']
            unsafe_hex_ids = [f['properties']['h3_index'] for f in unsafe_list]
            
            # Get overlapping constraint areas from all sources (OPTIMIZED: limit to 50 per dataset)
            combined = unsafe_fc.geometry()
            overlapping_protected = protected_areas.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_sssi = sssi_areas.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_sac = sac_areas.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_spa = spa_areas.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_ancient_woodland = ancient_woodland.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_ramsar = ramsar_sites.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_nnr = national_nature_reserves.filterBounds(combined).limit(50).getInfo()['features']
            overlapping_areas = overlapping_protected + overlapping_sssi + overlapping_sac + overlapping_spa + overlapping_ancient_woodland + overlapping_ramsar + overlapping_nnr
            
            # Simplified: Store area names once for all hexagons (OPTIMIZED: avoid per-hex processing)
            area_names = []
            for area in overlapping_areas[:20]:  # Limit to first 20 areas
                props = area['properties']
                name = (props.get('Name') or props.get('NAME') or props.get('name') or 
                       props.get('desig') or props.get('designation') or 
                       props.get('id') or "Unnamed Feature")
                area_names.append(name[:50])  # Truncate long names
            
            # Store constrained hexagon data (SIMPLIFIED: all hexagons share same constraint list)
            for hex_id in unsafe_hex_ids:
                analyzed_hexagons['all_hexagons'].append({
                    'h3_index': hex_id,
                    'hex_id': hex_id,
                    'is_unsafe': True,
                    'overlapping_areas': area_names[:5]  # Store only first 5
                })
            
            # Build HTML for constrained hexagons (no individual header - will be added later)
            summary_html = ""
            
            # Constrained hexagons section (COLLAPSIBLE - closed by default)
            hex_list_html = f"""
            <details style='background:#fff3cd; padding:10px; margin:5px 0; border-radius:4px; border-left:3px solid #4a4a4a;'>
                <summary style='cursor:pointer; font-weight:bold; margin-bottom:5px;'>
                    Constrained Hexagon IDs 
                    <span style='background:#4a4a4a; color:white; padding:2px 6px; border-radius:3px; font-size:11px;'>{len(unsafe_hex_ids)}</span>
                    {f'<span style="color:#dc3545; font-size:10px;"> (showing first {display_limit})</span>' if bad > display_limit else ''}
                    <span style='font-size:10px; color:#666;'>(click to expand)</span>
                </summary>
                <div style='font-family:monospace; font-size:11px; max-height:200px; overflow-y:auto; overflow-x:hidden; margin-top:5px; padding:8px; background:white; border-radius:3px; border:1px solid #4a4a4a;'>
            """
            for hex_id in unsafe_hex_ids:
                hex_list_html += f"• {hex_id}<br>"
            hex_list_html += "</div></details>"
            
            # Protected areas section (COLLAPSIBLE - closed by default)
            protected_html = f"""
            <details style='background:#f8d7da; padding:10px; margin:5px 0; border-radius:4px; border-left:3px solid #dc3545;'>
                <summary style='cursor:pointer; font-weight:bold; margin-bottom:5px;'>
                    ⚠️ {len(overlapping_areas)} CONSTRAINT AREAS FOUND
                    <span style='font-size:10px; color:#666;'>(click to expand)</span>
                </summary>
                <div style='background:white; padding:10px; margin-top:5px; border-radius:4px; border:1px solid #dc3545; max-height:200px; overflow-y:auto; overflow-x:hidden;'>
            """
            
            for area in overlapping_areas:
                props = area['properties']
                
                # Try to get name from various property fields
                name = (props.get('Name') or props.get('NAME') or props.get('name') or 
                       props.get('desig') or props.get('designation') or 
                       props.get('id') or "Unnamed Feature")
                
                # Get ID if available
                feature_id = props.get('id') or props.get('ID') or props.get('objectid') or props.get('OBJECTID') or ''
                if feature_id:
                    id_display = f"<span style='font-family:monospace; font-size:10px; color:#666;'>({feature_id})</span>"
                else:
                    id_display = ""
                
                protected_html += f"<div style='padding:5px; border-bottom:1px solid #eee;'>• <b>{name}</b> {id_display}</div>"
            
            protected_html += "</div></details>"
            
            # Combine constraint sections
            constraint_html = summary_html + hex_list_html + protected_html
        else:
            constraint_html = ""
        
        # BNG ANALYSIS FOR SAFE HEXAGONS (OPTIMIZED: process fewer hexagons)
        bng_html = ""
        if safe_count > 0 and safe_fc_with_alc:
            progress_label.value = (
                "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
                "<b>⏳ Analyzing BNG opportunities...</b><br>"
                "<span style='font-size:11px;'>Evaluating safe hexagons</span>"
                "</div>"
            )
            
            # Get safe hexagons with ALC data (OPTIMIZED: limit for UI performance)
            display_limit = min(safe_count, 300)  # Limit to 300 hexagons for display
            safe_list = safe_fc_with_alc.limit(display_limit).getInfo()['features']
            
            # Analyze BNG opportunities using 3-factor scoring model
            bng_analysis = {}
            hex_details = []
            
            for feat in safe_list:
                props = feat['properties']
                hex_id = props.get('h3_index', 'Unknown')
                alc_grades = props.get('alc_grades', 'No Data')
                habitat_classes = props.get('habitat_classes', '')
                has_chalk_river = props.get('has_chalk_river', False)
                
                # Get primary ALC grade (use first if multiple)
                primary_grade = 'No Data'
                if alc_grades and alc_grades != 'No Data':
                    grades_list = [g.strip() for g in str(alc_grades).split(',')]
                    primary_grade = grades_list[0] if grades_list else 'No Data'
                
                # Get primary habitat class (use first if multiple)
                primary_habitat = ''
                if habitat_classes and habitat_classes.strip():
                    habitat_list = [h.strip() for h in str(habitat_classes).split(',')]
                    primary_habitat = habitat_list[0] if habitat_list else ''
                
                # Calculate BNG score using 3-factor model
                score_result = calculate_bng_score(primary_grade, primary_habitat, has_chalk_river)
                bng_percentage, rating_text, color, score_loc, score_soil, score_water = score_result
                
                # Build breakdown text
                breakdown_parts = []
                if score_loc > 1:
                    breakdown_parts.append(f"Location: {score_loc}pts")
                if score_soil > 0:
                    breakdown_parts.append(f"Soil: {score_soil}pts")
                if score_water > 0:
                    breakdown_parts.append(f"Water: +{score_water}pts")
                breakdown = " | ".join(breakdown_parts) if breakdown_parts else "Base"
                
                # Store analysis
                rating_key = f"{rating_text} ({bng_percentage}%)"
                if rating_key not in bng_analysis:
                    bng_analysis[rating_key] = []
                bng_analysis[rating_key].append(hex_id)
                
                # Store hexagon detail for later display
                hex_detail = {
                    'h3_index': hex_id,
                    'hex_id': hex_id,
                    'grades': alc_grades,
                    'alc_grades': alc_grades,
                    'habitat_classes': habitat_classes,
                    'rating_score': bng_percentage,
                    'rating_text': rating_text,
                    'color': color,
                    'score_breakdown': breakdown,
                    'score_loc': score_loc,
                    'score_soil': score_soil,
                    'score_water': score_water,
                    'is_unsafe': False,
                    'has_chalk_river': has_chalk_river
                }
                hex_details.append(hex_detail)
                analyzed_hexagons['all_hexagons'].append(hex_detail)
            
            # Build results HTML
            bng_html = ""
            
            # Summary by rating
            if bng_analysis:
                bng_html += "<div style='background:white; padding:10px; margin:5px 0; border-radius:4px; border:1px solid #ddd;'>"
                bng_html += "<b>Summary by BNG Rating:</b><br>"
                for rating, hexes in sorted(bng_analysis.items(), key=lambda x: float(x[0].split('(')[1].split('%')[0]), reverse=True):
                    bng_html += f"<div style='padding:5px; margin:3px 0; background:#f8f9fa; border-radius:3px;'>"
                    bng_html += f"• <b>{rating}:</b> {len(hexes)} hexagons</div>"
                bng_html += "</div>"
            
            # Detailed hexagon list (COLLAPSIBLE - closed by default)
            bng_html += f"""
            <details style='background:#fff3cd; padding:10px; margin:5px 0; border-radius:4px; border-left:3px solid #ffc107;'>
                <summary style='cursor:pointer; font-weight:bold; margin-bottom:5px;'>
                    Detailed Hexagon Analysis
                    <span style='background:#ffc107; color:white; padding:2px 6px; border-radius:3px; font-size:11px;'>{len(hex_details)}</span>
                    {f'<span style="color:#dc3545; font-size:10px;"> (showing first {display_limit})</span>' if safe_count > display_limit else ''}
                    <span style='font-size:10px; color:#666;'>(click to expand)</span>
                </summary>
                <div style='font-size:11px; max-height:300px; overflow-y:auto; overflow-x:hidden; margin-top:5px; padding:8px; background:white; border-radius:3px; border:1px solid #ffc107;'>
            """
            
            # Sort by rating score (highest first)
            hex_details.sort(key=lambda x: x['rating_score'], reverse=True)
            
            for detail in hex_details:
                color_style = f"background:#{detail['color']}20; border-left:3px solid #{detail['color']};"
                breakdown = detail.get('score_breakdown', '')
                bng_html += f"""
                <div style='padding:6px; margin:3px 0; {color_style} border-radius:3px;'>
                    <div style='font-family:monospace; font-size:10px;'><b>{detail['hex_id']}</b></div>
                    <div style='font-size:10px;'><b style='color:#{detail['color']};'>{detail['rating_text']}: {detail['rating_score']}%</b></div>
                    <div style='font-size:9px; color:#666;'>{breakdown}</div>
                </div>
                """
            
            bng_html += "</div></details>"
            
            # Add BNG rating guide
            bng_html += """
            <div style='background:#f8f9fa; padding:10px; margin:5px 0; border-radius:4px; font-size:11px;'>
                <b>BNG Habitat Potential Percentage Model (0-100%):</b><br>
                <div style='padding:3px;'><b>📍 Location (1-10 pts, ~55% max):</b> Network Enhancement Zone priority</div>
                <div style='padding:3px;'><b>🌱 Soil (0-5 pts, ~28% max):</b> Agricultural Land Classification</div>
                <div style='padding:3px;'><b>💧 Water (+3 pts, ~17% max):</b> Proximity to Chalk Rivers (&lt;50m)</div>
                <div style='padding:3px; font-size:10px; color:#666;'><i>Formula: (Location + Soil + Water) / 18 × 100%</i></div>
                <hr style='margin:8px 0; border:none; border-top:1px solid #ddd;'>
                <b>Percentage Ranges:</b><br>
                <div style='padding:3px;'><span style='color:#9933ff;'>●</span> <b>Exceptional (≥83%):</b> Prime habitat creation sites</div>
                <div style='padding:3px;'><span style='color:#ff3333;'>●</span> <b>Very High (67-83%):</b> Excellent opportunity</div>
                <div style='padding:3px;'><span style='color:#ff8800;'>●</span> <b>High (50-67%):</b> Good potential</div>
                <div style='padding:3px;'><span style='color:#ffcc00;'>●</span> <b>Moderate (33-50%):</b> Fair opportunity</div>
                <div style='padding:3px;'><span style='color:#00cc66;'>●</span> <b>Low (17-33%):</b> Limited potential</div>
                <div style='padding:3px;'><span style='color:#0066ff;'>●</span> <b>Very Low (0-17%):</b> Minimal suitability</div>
            </div>
            """
        
        # FINAL: Combine constraint and BNG HTML with comprehensive summary at top
        overall_summary_html = f"""
        <div style='background:#f8f9fa; padding:15px; margin:5px 0; border-radius:6px; border:2px solid #495057;'>
            <div style='font-size:18px; font-weight:bold; margin-bottom:10px; color:#333;'>📊 ANALYSIS SUMMARY</div>
            <div style='display:grid; grid-template-columns:1fr 1fr; gap:10px; margin-top:10px;'>
                <div style='background:{'#dee2e6' if bad > 0 else '#d4edda'}; padding:10px; border-radius:4px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:{'#4a4a4a' if bad > 0 else '#28a745'};'>{bad}</div>
                    <div style='font-size:11px; color:#666;'>HEXAGONS CONSTRAINED</div>
                </div>
                <div style='background:#d4edda; padding:10px; border-radius:4px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#28a745;'>{safe_count}</div>
                    <div style='font-size:11px; color:#666;'>SAFE HEXAGONS</div>
                </div>
            </div>
            <div style='background:white; padding:10px; margin-top:10px; border-radius:4px; text-align:center;'>
                <div style='font-size:14px; font-weight:bold;'>{bad} OF {total} HEXAGONS CONSTRAINED ({pct}%)</div>
            </div>
        </div>
        """
        
        # Add section headers
        if bad > 0:
            constraint_section_header = """
            <div style='background:#495057; color:white; padding:10px; margin:10px 0 5px 0; border-radius:4px;'>
                <b style='font-size:14px;'>⚠️ CONSTRAINT DETAILS</b>
            </div>
            """
            constraint_html = constraint_section_header + constraint_html
        
        if safe_count > 0:
            bng_section_header = """
            <div style='background:#495057; color:white; padding:10px; margin:10px 0 5px 0; border-radius:4px;'>
                <b style='font-size:14px;'>🌱 BNG OPPORTUNITY ANALYSIS</b>
            </div>
            """
            bng_html = bng_section_header + bng_html
        
        final_html = overall_summary_html + constraint_html + bng_html
        if final_html:
            results_panel.children = [widgets.HTML(final_html)]
            # Show the panel after analysis completes
            panel.layout.display = 'flex'
        
        # Final status update
        if bad == 0:
            progress_label.value = (
                "<div style='padding:8px; background:#d4edda; border-radius:4px; border-left:3px solid #28a745;'>"
                f"<b>✅ Analysis Complete</b><br>"
                f"<span style='font-size:11px;'>All {total} hexagons are safe</span>"
                "</div>"
            )
        else:
            progress_label.value = (
                "<div style='padding:8px; background:#fff3cd; border-radius:4px; border-left:3px solid #ffc107;'>"
                f"<b>✅ Analysis Complete</b><br>"
                f"<span style='font-size:11px;'>{bad} of {total} hexagons constrained ({pct}%)</span>"
                "</div>"
            )
        
        # Show clear button after analysis
        clear_button.layout.display = 'block'

    except Exception as e:
        # Format error for HTML display
        err_msg = str(e).replace('<', '&lt;').replace('>', '&gt;')
        progress_label.value = (
            "<div style='padding:8px; background:#f8d7da; border-radius:4px; border-left:3px solid #dc3545;'>"
            f"<b>❌ Error</b><br><span style='font-size:11px;'>{err_msg[:100]}</span>"
            "</div>"
        )
        results_panel.children = []

# 4. DRAWING TOOL
draw_control = DrawControl(
    polygon={'shapeOptions': {'color': '#3388ff'}},
    rectangle={'shapeOptions': {'color': '#3388ff'}},
    circlemarker={}, polyline={}
)

def handle_draw(target, action, geo_json):
    if action == 'created':
        draw_control.clear()
        run_analysis(geo_json['geometry'])

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Add click handler for hexagon inspection
# Store analyzed hexagons data globally for click access
analyzed_hexagons = {}
current_hex_detail = None  # Track current hexagon detail widget control
highlighted_hex_layer = None  # Track highlighted hexagon layer name
last_clicked_hex_id = None  # Track last clicked hexagon for toggle

def handle_hexagon_click(**kwargs):
    """Handle clicks on hexagons to show detailed info"""
    global current_hex_detail, highlighted_hex_layer, last_clicked_hex_id
    
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        
        # Only process if we have analyzed hexagons
        if not analyzed_hexagons:
            return
        
        # Find which hexagon was clicked
        res = analyzed_hexagons.get('resolution')
        if not res:
            return
            
        clicked_hex_id = h3.latlng_to_cell(lat, lon, res)
        
        # Toggle: if clicking the same hexagon, close the detail panel
        if clicked_hex_id == last_clicked_hex_id:
            # Remove detail widget
            if current_hex_detail:
                m.remove_control(current_hex_detail)
                current_hex_detail = None
            
            # Remove highlight layer
            if highlighted_hex_layer and m.find_layer(highlighted_hex_layer):
                m.remove_layer(m.find_layer(highlighted_hex_layer))
                highlighted_hex_layer = None
            
            last_clicked_hex_id = None
            return
        
        # Check if this hexagon is in our analyzed set
        hex_data = None
        for hex_info in analyzed_hexagons.get('all_hexagons', []):
            if hex_info['h3_index'] == clicked_hex_id:
                hex_data = hex_info
                break
        
        if not hex_data:
            # Click was outside analyzed hexagons - close detail panel
            if current_hex_detail:
                m.remove_control(current_hex_detail)
                current_hex_detail = None
            if highlighted_hex_layer and m.find_layer(highlighted_hex_layer):
                m.remove_layer(m.find_layer(highlighted_hex_layer))
                highlighted_hex_layer = None
            last_clicked_hex_id = None
            return
        
        # Build detailed info HTML for this hexagon (COLLAPSIBLE)
        info_html = f"""
        <details open style='background:#495057; margin:5px 0; border-radius:4px;'>
            <summary style='cursor:pointer; color:white; font-weight:bold; padding:10px;'>
                📍 HEXAGON DETAILS
                <span style='font-size:10px; opacity:0.9;'>(click hexagon again or elsewhere to close)</span>
            </summary>
            <div style='background:white; padding:12px; border-radius:0 0 4px 4px; border:1px solid #495057; border-top:none;'>
                <div style='font-family:monospace; font-size:11px; padding:5px; background:#f8f9fa; border-radius:3px; margin-bottom:8px;'>
                    <b>H3 Index:</b><br>{clicked_hex_id}
                </div>
        """
        
        # Status
        if hex_data.get('is_unsafe'):
            info_html += """
            <div style='background:#e9ecef; padding:8px; margin:5px 0; border-radius:3px; border-left:3px solid #4a4a4a;'>
                <b>⚠️ Status: CONSTRAINED</b><br>
                <span style='font-size:11px;'>Overlaps with environmental/heritage constraints</span>
            </div>
            """
            
            # Show which protected areas
            overlapping_areas = hex_data.get('overlapping_areas', [])
            if overlapping_areas:
                info_html += "<div style='margin-top:8px;'><b>Constraint Areas:</b></div>"
                for area_name in overlapping_areas[:5]:  # Limit to 5
                    info_html += f"<div style='padding:3px; font-size:11px;'>• {area_name}</div>"
                if len(overlapping_areas) > 5:
                    info_html += f"<div style='padding:3px; font-size:11px; color:#666;'>... and {len(overlapping_areas)-5} more</div>"
        else:
            # Safe hexagon
            rating_score = hex_data.get('rating_score', 5)
            rating_text = hex_data.get('rating_text', 'Unknown')
            color = hex_data.get('color', '6c757d')
            
            # Status with BNG color
            info_html += f"""
            <div style='background:#{color}20; padding:8px; margin:5px 0; border-radius:3px; border-left:3px solid #{color};'>
                <b>✅ Status: SAFE - {rating_text} BNG Potential</b><br>
                <span style='font-size:11px;'>No environmental/heritage constraint overlap</span>
            </div>
            """
            
            # Show BNG Score Breakdown with class names and explanations
            score_breakdown = hex_data.get('score_breakdown', '')
            score_loc = hex_data.get('score_loc', 0)
            score_soil = hex_data.get('score_soil', 0)
            score_water = hex_data.get('score_water', 0)
            habitat_classes = hex_data.get('habitat_classes', '')
            alc_grades = hex_data.get('alc_grades', 'No Data')
            
            info_html += f"""
            <div style='margin-top:8px; padding:10px; background:#{color}20; border-radius:3px; border-left:3px solid #{color};'>
                <b>BNG Habitat Potential: <span style='color:#{color}; font-size:16px;'>{rating_score}%</span></b>
                <div style='margin-top:10px; font-size:11px; line-height:1.8;'>
            """
            
            # Location score with class name and explanation
            if score_loc > 1 and habitat_classes:
                primary_habitat = habitat_classes.split(',')[0].strip()
                explanation = get_habitat_classification_explanation(primary_habitat)
                # Remove HTML tags for cleaner display
                clean_explanation = explanation.replace('🟢 <b>', '').replace('🟡 <b>', '').replace('🟠 <b>', '').replace('🔵 <b>', '').replace('⭐ <b>', '').replace('💚 <b>', '').replace('🚨 <b>', '').replace('🌍 <b>', '').replace('ℹ️ <b>', '').replace('🌳 <b>', '').replace('</b>', '').replace('<b>', '')
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>📍 Location: {score_loc} pts</div>
                        <div style='margin-left:10px; color:#333; font-size:10px;'>
                            <div style='font-weight:bold; color:#0066cc; margin-bottom:2px;'>{primary_habitat}</div>
                            <div style='color:#666;'>{clean_explanation}</div>
                        </div>
                    </div>
                """
            else:
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>📍 Location: {score_loc} pt</div>
                        <div style='margin-left:10px; color:#666; font-size:10px;'>No priority habitat network classification</div>
                    </div>
                """
            
            # Soil score with ALC grade and explanation
            if score_soil > 0 and alc_grades != 'No Data':
                primary_grade = alc_grades.split(',')[0].strip()
                if score_soil == 5:
                    soil_explanation = "Poor agricultural quality - ideal for biodiversity habitat creation"
                elif score_soil == 2:
                    soil_explanation = "Moderate agricultural quality - suitable for habitat with planning"
                else:
                    soil_explanation = "Lower suitability for habitat conversion"
                    
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>🌱 Soil: {score_soil} pts</div>
                        <div style='margin-left:10px; color:#333; font-size:10px;'>
                            <div style='font-weight:bold; color:#0066cc; margin-bottom:2px;'>{primary_grade}</div>
                            <div style='color:#666;'>{soil_explanation}</div>
                        </div>
                    </div>
                """
            else:
                soil_reason = "High-value agricultural land - preserve for food production" if alc_grades != 'No Data' else "No agricultural classification data"
                grade_display = f"<div style='font-weight:bold; color:#0066cc; margin-bottom:2px;'>{alc_grades.split(',')[0].strip()}</div>" if alc_grades != 'No Data' else ""
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>🌱 Soil: {score_soil} pts</div>
                        <div style='margin-left:10px; font-size:10px;'>
                            {grade_display}
                            <div style='color:#666;'>{soil_reason}</div>
                        </div>
                    </div>
                """
            
            # Water bonus with explanation
            if score_water > 0:
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>💧 Water: +{score_water} pts</div>
                        <div style='margin-left:10px; color:#333; font-size:10px;'>
                            <div style='font-weight:bold; color:#0066cc; margin-bottom:2px;'>Chalk River Proximity</div>
                            <div style='color:#666;'>Protected chalk stream habitat nearby - excellent for water-dependent species and connectivity</div>
                        </div>
                    </div>
                """
            else:
                info_html += f"""
                    <div style='margin-bottom:10px; padding:8px; background:white; border-radius:3px;'>
                        <div style='font-weight:bold; margin-bottom:4px;'>💧 Water: {score_water} pts</div>
                        <div style='margin-left:10px; color:#666; font-size:10px;'>No chalk river proximity</div>
                    </div>
                """
            
            info_html += f"""
                </div>
                <div style='padding-top:8px; border-top:1px solid rgba(0,0,0,0.1); text-align:center;'>
                    <span style='font-size:12px; font-weight:bold;'>Overall Rating: </span>
                    <span style='font-size:14px; font-weight:bold; color:#{color};'>{rating_text}</span>
                </div>
            </div>
            """
            
            # Show ALC and habitat info
            alc_grades = hex_data.get('alc_grades', 'No Data')
            if alc_grades and alc_grades != 'No Data':
                info_html += f"""
                <div style='margin-top:8px; padding:8px; background:#fff3cd; border-radius:3px;'>
                    <b>Agricultural Land:</b><br>
                    <span style='font-size:11px;'>{alc_grades}</span>
                </div>
                """
                
                # Show BNG opportunity enhancements if present
                has_habitat_network = hex_data.get('has_habitat_network', False)
                has_chalk_river = hex_data.get('has_chalk_river', False)
                
                if has_habitat_network or has_chalk_river:
                    info_html += """
                    <div style='margin-top:8px; padding:8px; background:#e7f3ff; border-radius:3px; border-left:3px solid #0088ff;'>
                        <b>🌟 BNG Opportunity Enhancements:</b><br>
                    """
                    
                    if has_habitat_network:
                        habitat_classes = hex_data.get('habitat_classes', '')
                        
                        if habitat_classes and habitat_classes.strip():
                            # Parse multiple classifications if present
                            classifications = [c.strip() for c in habitat_classes.split(',') if c.strip()]
                            
                            for classification in classifications:
                                # Show each classification as a collapsible section
                                info_html += f"""
                                <details style='margin-top:5px;'>
                                    <summary style='cursor:pointer; font-weight:bold; color:#0088ff; font-size:11px;'>
                                        ✓ {classification} <span style='font-size:9px; color:#666;'>(click for details)</span>
                                    </summary>
                                    <div style='margin-top:5px; padding:8px; background:white; border-radius:3px; font-size:10px;'>
                                """
                                
                                # Get and display explanation
                                explanation = get_habitat_classification_explanation(classification)
                                info_html += f"<div style='padding:5px; line-height:1.4;'>{explanation}</div>"
                                
                                info_html += "</div></details>"
                        else:
                            info_html += """
                            <div style='margin-top:5px; font-size:11px;'>
                                ✓ Within National Habitat Network <span style='color:#666;'>(classification data loading...)</span>
                            </div>
                            """
                    
                    if has_chalk_river:
                        info_html += "<div style='margin-top:5px; font-size:11px;'>✓ Near Chalk River</div>"
                    
                    info_html += "</div>"
        
        info_html += "</div></details>"
        
        # Remove previous hexagon detail if exists
        if current_hex_detail:
            m.remove_control(current_hex_detail)
        
        # Remove previous highlight layer if exists
        if highlighted_hex_layer and m.find_layer(highlighted_hex_layer):
            m.remove_layer(m.find_layer(highlighted_hex_layer))
        
        # Create highlight layer for clicked hexagon
        hex_boundary = h3.cell_to_boundary(clicked_hex_id)
        hex_coords = [[p[1], p[0]] for p in hex_boundary]  # Flip to [lon, lat]
        hex_coords.append(hex_coords[0])  # Close the polygon
        hex_geom = ee.Geometry.Polygon([hex_coords])
        hex_feature = ee.Feature(hex_geom, {'h3_index': clicked_hex_id})
        hex_fc = ee.FeatureCollection([hex_feature])
        
        # Add highlight with bright yellow border
        highlighted_hex_layer = f'Highlighted_{clicked_hex_id}'
        m.add_layer(hex_fc.style(color='FFFF00', width=4, fillColor='FFFF0000'), {}, highlighted_hex_layer)
        
        # Create new hexagon detail widget (no close button - use toggle behavior)
        detail_widget = widgets.HTML(
            info_html,
            layout=widgets.Layout(
                width='600px',
                max_height='60vh',  # Maximum 60% of viewport height
                overflow_y='auto',  # Enable scrolling
                overflow_x='hidden',  # Prevent horizontal scroll
                padding='10px',
                border='2px solid #495057',  # Add visible border
                border_radius='6px',
                background_color='#ffffff'  # White background
            )
        )
        
        # Create widget control at bottom left (as close to center as possible)
        current_hex_detail = WidgetControl(widget=detail_widget, position='bottomleft')
        m.add_control(current_hex_detail)
        
        # Update last clicked hex
        last_clicked_hex_id = clicked_hex_id

m.on_interaction(handle_hexagon_click)

# 5. UI - ENHANCED INFO PANEL
# Progress indicator
progress_label = widgets.HTML(
    "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
    "<b>📍 Ready to Analyze</b><br>"
    "<span style='font-size:11px;'>Select Pentagon ⬠ tool and draw a shape to begin</span>"
    "</div>"
)

info_label = widgets.HTML(
    "<div style='padding:10px; background:#f8f9fa; border-radius:4px;'>"
    "<b>Instructions:</b><br>"
    "1. Select Pentagon ⬠ tool and draw a shape<br>"
    "2. Wait for analysis to complete<br>"
    "3. <b>Click any hexagon</b> to see its details"
    "</div>"
)

# Results panel (will be populated after analysis)
results_panel = widgets.VBox([])

# Clear button
clear_button = widgets.Button(
    description='🗑️ Clear Results',
    button_style='warning',
    layout=widgets.Layout(width='auto', display='none')
)

def clear_results(b):
    """Clear all analysis results and layers"""
    global analyzed_hexagons, current_hex_detail, highlighted_hex_layer, last_clicked_hex_id
    
    # Remove all analysis layers
    for layer in ['Constraints Found', 'Constraint Borders', 'ALC Overlapping',
                  'Habitat Networks (Overlapping)', 'Chalk Rivers (Overlapping)',
                  'Exceptional BNG (≥83%, Purple)', 'Very High BNG (67-83%, Red)', 
                  'High BNG (50-67%, Orange)', 'Moderate BNG (33-50%, Yellow)',
                  'Low BNG (17-33%, Green)', 'Very Low BNG (0-17%, Blue)']:
        if m.find_layer(layer):
            m.remove_layer(m.find_layer(layer))
    
    # Remove highlight layer if exists
    if highlighted_hex_layer and m.find_layer(highlighted_hex_layer):
        m.remove_layer(m.find_layer(highlighted_hex_layer))
    
    # Remove hexagon detail widget control if exists
    if current_hex_detail:
        m.remove_control(current_hex_detail)
        current_hex_detail = None
    
    results_panel.children = []
    clear_button.layout.display = 'none'
    analyzed_hexagons.clear()  # Clear click handler data
    current_hex_detail = None  # Reset hexagon detail widget
    highlighted_hex_layer = None  # Reset highlighted layer
    last_clicked_hex_id = None  # Reset last clicked hex
    
    # Hide the panel
    panel.layout.display = 'none'
    
    # Show instructions again
    info_label.layout.display = 'flex'
    
    # Reset progress
    progress_label.value = (
        "<div style='padding:8px; background:#e7f3ff; border-radius:4px; border-left:3px solid #0066cc;'>"
        "<b>📍 Ready to Analyze</b><br>"
        "<span style='font-size:11px;'>Select Pentagon ⬠ tool and draw a shape to begin</span>"
        "</div>"
    )

clear_button.on_click(clear_results)

# Container with proper styling - visible by default with instructions
panel = widgets.VBox(
    [progress_label, info_label, clear_button, results_panel],
    layout=widgets.Layout(
        width='420px',
        max_height='70vh',  # Reduced to 70% to ensure it stays within viewport
        overflow_y='auto',  # Vertical scrollbar
        overflow_x='hidden',
        padding='10px',
        border='2px solid #495057',  # Dark grey border for clear frame
        border_radius='6px',
        background_color='#f8f9fa',  # Light background
        display='flex'  # Visible by default to show instructions
    )
)

control = WidgetControl(widget=panel, position='topright')  # Changed to topright for more space
m.add_control(control)

# Set map height for better display
m.layout.height = '900px'  # Increased from 800px
m.layout.width = '100%'    # Full width of notebook cell

m